In [1]:
import pandas as pd
import numpy as np
import os
import re

BASE_DIR = os.getcwd()
DIR_NAME = os.path.basename(BASE_DIR)
print(BASE_DIR, DIR_NAME)
while DIR_NAME != "RakutenTeam": 
    BASE_DIR = os.path.realpath(os.path.join(os.path.dirname(__name__), '..'))
    DIR_NAME = os.path.basename(BASE_DIR)
    os.chdir(BASE_DIR)
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    print( BASE_DIR, DATA_DIR, DIR_NAME)


c:\Users\mangg\projects\RakutenTeam\notebook\tests tests
C:\Users\mangg\projects\RakutenTeam\notebook C:\Users\mangg\projects\RakutenTeam\notebook\data notebook
C:\Users\mangg\projects\RakutenTeam C:\Users\mangg\projects\RakutenTeam\data RakutenTeam


In [2]:
from src.data_loader import import_data

folder_path = os.path.join(DATA_DIR, "raw")
print(folder_path)

C:\Users\mangg\projects\RakutenTeam\data\raw


In [3]:
df = import_data(folder_path=folder_path)

In [4]:
designations = df.designation
descriptions = df.description

In [5]:
from src.features.text.pipelines.cleaner import CleanTextPipeline

pipe = CleanTextPipeline()
cleaned_descriptions = pipe.fit_transform(descriptions)

C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:24: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:47: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')
C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:47: MarkupResemblesLocatorWarn

In [6]:

pattern1 = r'([?¿º¢©́])\1'
pattern2 = r'[å¿]'
pattern3 = r"[?\s:;\/\!]{3,}"
pattern4 = r'\w*[å¿]\w*'


In [7]:
cleaned_descriptions.str.count(pattern1).sum(), cleaned_descriptions.str.count(pattern2).sum(), cleaned_descriptions.str.count(pattern3).sum(), cleaned_descriptions.str.count(pattern4).sum()

(8487.0, 13180.0, 132837.0, 13180.0)

In [8]:
from src.features.text.pipelines.corrector import CleanEncodingPipeline

encoding_pipeline = CleanEncodingPipeline()

corrected_descriptions = encoding_pipeline.fit_transform(cleaned_descriptions)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [9]:
corrected_descriptions.str.count(pattern1).sum(), corrected_descriptions.str.count(pattern2).sum(), corrected_descriptions.str.count(pattern3).sum(),  corrected_descriptions.str.count(pattern4).sum()


(0.0, 6013.0, 0.0, 6013.0)

In [10]:
k = 5
selected_filter = corrected_descriptions.str.count(pattern2) > 0

In [11]:
phrase = corrected_descriptions[selected_filter].iloc[k]

search_pattern = r'\w*[å¿]\w*'
#search_pattern2 = r"[\S]*[å¿][\S^\.]*"

r = re.compile(search_pattern)

r.findall(phrase)

['Caract¿ristique', 'durabilit¿', '¿paisse']

In [12]:
phrase

'Taille: En format A5 (144 cm x 21 cm) Caract¿ristique: -Excellente durabilit¿ avec couverture solide design. 96 pages jaune pale (recto et verso) -Texture de papier de haute qualit . crire facilement sur et assez ¿paisse.'

In [13]:
from src.features.text.transformers.encoding_corrector import BadWordsDetector

detector = BadWordsDetector()
bad_words_df = detector.fit_transform(corrected_descriptions)

In [14]:
pattern_filter = corrected_descriptions.str.count(search_pattern) > 0
#bad_words[pattern_filter]
#new_df = pd.DataFrame({"description": corrected_descriptions, "bad_words": bad_words})

In [15]:
bad_words_df[pattern_filter]

,text,badwords,lang
14,MATELAS: Â· Accueil . Ferme . Â· Soutien . Trè...,"[Cà¿tés, dâ¿hygiène, à¿me]",fr
24,Mini Wifi 720P Caméra Drone RC Quadcopter 24 G...,"[¿uvre, ¿uvre, ¿uvre]",fr
33,Timeless elegance a clear design versatility f...,"[crÃ¿me, crÃ¿me]",en
68,Journal littéraire scientifique et religieux -...,"[¿Notre, ¿Croquis]",fr
84,Bâche hiver pour piscine Toledo - 280 g/m² - O...,[¿illet],fr
...,...,...,...
84416,Cette cartouche filtrante 29003 de la marque I...,"[tr¿s, Diam¿tre]",fr
84546,- Time to play - hiver à partir de 5 ans - 2 j...,[L¿air],fr
84677,Silicone Demi-paquet protecteur Couleur pure H...,"[r¿sistant, salet¿, Fabriqu¿, prot¿ger, poign¿...",fr
84690,Meuble TV mural en bois de frêne qui offre une...,[etc¿],fr


In [16]:
from src.features.text.transformers.encoding_corrector import BadWordsCorrector

corrector = BadWordsCorrector()

corrector.fit_transform(bad_words_df)

<class 'pandas.core.frame.DataFrame'>
Index(['text', 'badwords', 'lang'], dtype='object')


KeyError: 'text'

In [ ]:
s = pd.Series({"a": "a", "c": np.nan})

In [ ]:
s.str.join("")

a      a
c    NaN
dtype: object